In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Asthma"
cohort = "GSE123086"

# Input paths
in_trait_dir = "../../input/GEO/Asthma"
in_cohort_dir = "../../input/GEO/Asthma/GSE123086"

# Output paths
out_data_file = "../../output/preprocess/Asthma/GSE123086.csv"
out_gene_data_file = "../../output/preprocess/Asthma/gene_data/GSE123086.csv"
out_clinical_data_file = "../../output/preprocess/Asthma/clinical_data/GSE123086.csv"
json_path = "../../output/preprocess/Asthma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A validated single-cell-based strategy to identify diagnostic and therapeutic targets in complex diseases [study of 13 diseases]"
!Series_summary	"We conducted prospective clinical studies to validate the importance of CD4+ T cells in 13 diseases from the following ICD-10-CM chapters: Neoplasms (breast cancer, chronic lymphocytic leukemia); endocrine, nutritional and metabolic diseases (type I diabetes, obesity); diseases of the circulatory system (atherosclerosis); diseases of the respiratory system (acute tonsillitis, influenza, seasonal allergic rhinitis, asthma); diseases of the digestive system (Crohn’s disease [CD], ulcerative colitis [UC]); and diseases of the skin and subcutaneous tissue (atopic eczema, psoriatic diseases)."
!Series_summary	"Study participants were recruited by clinical specialists based on diagnostic criteria defined by organizations representing each specialist’s discipline. Age and gender matched healthy controls (n = 1

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from CD4+ T cells
# analyzed through microarrays (Agilent)
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For the trait (Asthma)
# Looking at index 1 for primary diagnosis, which includes ASTHMA
trait_row = 1

# For age
# Looking at indexes 3 and 4, which contain age values
age_row = 3  # Primary age row

# For gender
# Looking at indices 2 and 3, both contain gender information
gender_row = 2  # Primary gender row

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert trait value to binary (0=control, 1=Asthma)"""
    if value is None or pd.isna(value):
        return None
    
    # Extract value part after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Check if the value indicates Asthma
    if "ASTHMA" in value.upper():
        return 1
    elif "HEALTHY_CONTROL" in value.upper():
        return 0
    else:
        return None  # Other diseases

def convert_age(value):
    """Convert age value to continuous numeric value"""
    if value is None or pd.isna(value):
        return None
    
    # Extract value part after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender value to binary (0=female, 1=male)"""
    if value is None or pd.isna(value):
        return None
    
    # Extract value part after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "MALE" in value.upper():
        return 1
    elif "FEMALE" in value.upper():
        return 0
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering on dataset usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # For this step, we'd typically load clinical_data from a file
    # But since we don't have access to that file directly,
    # we'll create a placeholder for now, and the actual processing
    # will be done in a subsequent step once we have the actual data
    
    # Simply log that we've completed the identification phase
    print(f"Successfully identified data availability for {cohort}:")
    print(f"- Trait data available at row {trait_row}")
    print(f"- Age data available at row {age_row}")
    print(f"- Gender data available at row {gender_row}")
    print("Clinical data processing will be performed in subsequent steps.")


Successfully identified data availability for GSE123086:
- Trait data available at row 1
- Age data available at row 3
- Gender data available at row 2
Clinical data processing will be performed in subsequent steps.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Asthma/GSE123086/GSE123086_series_matrix.txt.gz


Gene data shape: (22881, 166)
First 20 gene/probe identifiers:
Index(['1', '2', '3', '9', '10', '12', '13', '14', '15', '16', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Review the gene identifiers
# These appear to be numeric identifiers, not standard human gene symbols.
# Typically, human gene symbols are alphabetic (like BRCA1, TP53) or alphanumeric (like CD4, IL6).
# The identifiers shown are purely numeric, suggesting they're likely probe IDs or some other internal identifiers
# that need to be mapped to actual gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# Check if there are any platforms defined in the SOFT file that might contain annotation data
with gzip.open(soft_file, 'rt') as f:
    soft_content = f.read()

# Look for platform sections in the SOFT file
platform_sections = re.findall(r'^!Platform_title\s*=\s*(.+)$', soft_content, re.MULTILINE)
if platform_sections:
    print(f"Platform title found: {platform_sections[0]}")

# Try to extract more annotation data by reading directly from the SOFT file
# Look for lines that might contain gene symbol mappings
symbol_pattern = re.compile(r'ID_REF\s+Symbol|ID\s+Gene Symbol', re.IGNORECASE)
annotation_lines = []
with gzip.open(soft_file, 'rt') as f:
    for line in f:
        if symbol_pattern.search(line):
            annotation_lines.append(line)
            # Collect the next few lines to see the annotation structure
            for _ in range(10):
                annotation_lines.append(next(f, ''))

if annotation_lines:
    print("Found potential gene symbol mappings:")
    for line in annotation_lines:
        print(line.strip())

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("\nGene annotation preview:")
print(preview_df(gene_annotation, n=10))

# If we need an alternative source of mapping, check if there are any other annotation files in the cohort directory
cohort_files = os.listdir(in_cohort_dir)
annotation_files = [f for f in cohort_files if 'annotation' in f.lower() or 'platform' in f.lower()]
if annotation_files:
    print("\nAdditional annotation files found in the cohort directory:")
    for file in annotation_files:
        print(file)


Platform title found: Agilent-039494 SurePrint G3 Human GE v2 8x60K Microarray 039381 (Entrez Gene ID  version)



Gene annotation preview:
{'ID': ['1', '2', '3', '9', '10', '12', '13', '14', '15', '16'], 'ENTREZ_GENE_ID': ['1', '2', '3', '9', '10', '12', '13', '14', '15', '16'], 'SPOT_ID': [1.0, 2.0, 3.0, 9.0, 10.0, 12.0, 13.0, 14.0, 15.0, 16.0]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Analyze which columns in gene_annotation match the gene identifiers in gene_data
# From the platform title, we know this uses Entrez Gene IDs
# We need to extract these properly for mapping

# Check gene_annotation structure
print("Gene annotation columns:")
print(gene_annotation.columns.tolist())

# Create a proper mapping dataframe using Entrez Gene IDs
probe_col = 'ID'
gene_col = 'ENTREZ_GENE_ID'

# 2. Get the gene mapping dataframe by extracting the two relevant columns
gene_mapping = gene_annotation[[probe_col, gene_col]].copy()
gene_mapping.columns = ['ID', 'Gene']  # Rename to match required format for apply_gene_mapping
gene_mapping = gene_mapping.dropna()   # Remove rows with missing values
gene_mapping = gene_mapping.astype({'ID': 'str', 'Gene': 'str'})  # Ensure proper types

print(f"Gene mapping shape: {gene_mapping.shape}")
print("Gene mapping preview:")
print(gene_mapping.head())

# 3. Convert probe-level measurements to gene expression data using the library function
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Mapped gene expression data shape: {gene_data.shape}")

# Check if we have valid gene data
if gene_data.shape[0] > 0:
    print("First few gene identifiers after mapping:")
    print(gene_data.index[:10].tolist())
else:
    print("No genes were mapped. Checking for issues...")
    
    # Check overlap between probe IDs in expression data and mapping data
    expr_ids = set(gene_data.index.astype(str))
    map_ids = set(gene_mapping['ID'].astype(str))
    overlap = expr_ids.intersection(map_ids)
    
    print(f"Expression data has {len(expr_ids)} unique probe IDs")
    print(f"Mapping data has {len(map_ids)} unique probe IDs")
    print(f"Overlap between the two: {len(overlap)} probe IDs")
    
    # Sample overlapping IDs
    if overlap:
        print("Sample overlapping IDs:", list(overlap)[:5])


Gene annotation columns:
['ID', 'ENTREZ_GENE_ID', 'SPOT_ID']


Gene mapping shape: (3822578, 2)
Gene mapping preview:
   ID Gene
0   1    1
1   2    2
2   3    3
3   9    9
4  10   10


Mapped gene expression data shape: (0, 166)
No genes were mapped. Checking for issues...
Expression data has 0 unique probe IDs
Mapping data has 24167 unique probe IDs
Overlap between the two: 0 probe IDs


### Step 7: Gene Identifier Mapping

In [8]:
# First, let's reload the gene data to ensure we're working with the original structure
gene_data = get_genetic_data(matrix_file)
print(f"Original gene data shape: {gene_data.shape}")
print(f"Gene data index name: {gene_data.index.name}")
print(f"First few probe IDs: {gene_data.index[:5].tolist()}")

# In the previous step, the gene mapping data frame was created correctly
# The issue is with the probe IDs in the expression data vs. the mapping data
print("\nChecking gene mapping again:")
probe_col = 'ID'
gene_col = 'ENTREZ_GENE_ID'

# Get the gene mapping dataframe 
gene_mapping = gene_annotation[[probe_col, gene_col]].copy()
gene_mapping.columns = ['ID', 'Gene']
gene_mapping = gene_mapping.dropna()
gene_mapping = gene_mapping.astype({'ID': 'str', 'Gene': 'str'})

print(f"Gene mapping shape: {gene_mapping.shape}")
print(f"Gene mapping sample: {gene_mapping.head().to_dict('records')}")

# The issue might be that the gene_data's index is already named 'ID'
# Try to apply the mapping with the correct probe/gene relationship
gene_data_reset = gene_data.reset_index()  # This should create a column 'ID' with the probe identifiers
print(f"\nReset gene data columns: {gene_data_reset.columns.tolist()}")

# Set the index back to 'ID' to ensure proper functionality with apply_gene_mapping
gene_data_reset.set_index('ID', inplace=True)

# Now apply the gene mapping
gene_expression = apply_gene_mapping(gene_data_reset, gene_mapping)
print(f"\nMapped gene expression data shape: {gene_expression.shape}")

# Check the result
if gene_expression.shape[0] > 0:
    print("First few gene identifiers after mapping:")
    print(gene_expression.index[:10].tolist())
    # Update gene_data to contain the mapped expression data
    gene_data = gene_expression
else:
    print("Still no genes mapped. Let's debug more thoroughly.")
    # Check the first few IDs in both datasets to see the format difference
    print(f"Expression data first few IDs: {gene_data.index[:5].tolist()}")
    print(f"Mapping data first few IDs: {gene_mapping['ID'].head(5).tolist()}")
    
    # Try alternative mapping approach in case of formatting differences
    # Create a set with string-converted IDs from both datasets
    expr_ids_set = set(gene_data.index.astype(str).tolist())
    map_ids_set = set(gene_mapping['ID'].astype(str).tolist())
    overlap = expr_ids_set.intersection(map_ids_set)
    print(f"Overlap count after string conversion: {len(overlap)}")
    
    # If there's still an issue, we'll normalize the gene IDs directly using extract_human_gene_symbols
    # This uses the ENTREZ_GENE_ID which should contain gene identifiers
    gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"\nNormalized gene data shape: {gene_data.shape}")

Original gene data shape: (22881, 166)
Gene data index name: ID
First few probe IDs: ['1', '2', '3', '9', '10']

Checking gene mapping again:


Gene mapping shape: (3822578, 2)
Gene mapping sample: [{'ID': '1', 'Gene': '1'}, {'ID': '2', 'Gene': '2'}, {'ID': '3', 'Gene': '3'}, {'ID': '9', 'Gene': '9'}, {'ID': '10', 'Gene': '10'}]

Reset gene data columns: ['ID', 'GSM3494884', 'GSM3494885', 'GSM3494886', 'GSM3494887', 'GSM3494888', 'GSM3494889', 'GSM3494890', 'GSM3494891', 'GSM3494892', 'GSM3494893', 'GSM3494894', 'GSM3494895', 'GSM3494896', 'GSM3494897', 'GSM3494898', 'GSM3494899', 'GSM3494900', 'GSM3494901', 'GSM3494902', 'GSM3494903', 'GSM3494904', 'GSM3494905', 'GSM3494906', 'GSM3494907', 'GSM3494908', 'GSM3494909', 'GSM3494910', 'GSM3494911', 'GSM3494912', 'GSM3494913', 'GSM3494914', 'GSM3494915', 'GSM3494916', 'GSM3494917', 'GSM3494918', 'GSM3494919', 'GSM3494920', 'GSM3494921', 'GSM3494922', 'GSM3494923', 'GSM3494924', 'GSM3494925', 'GSM3494926', 'GSM3494927', 'GSM3494928', 'GSM3494929', 'GSM3494930', 'GSM3494931', 'GSM3494932', 'GSM3494933', 'GSM3494934', 'GSM3494935', 'GSM3494936', 'GSM3494937', 'GSM3494938', 'GSM349493


Mapped gene expression data shape: (0, 166)
Still no genes mapped. Let's debug more thoroughly.
Expression data first few IDs: ['1', '2', '3', '9', '10']
Mapping data first few IDs: ['1', '2', '3', '9', '10']
Overlap count after string conversion: 22881

Normalized gene data shape: (0, 166)
